Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [2]:
import os
import re
import time

import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    proper_date=(dt - timedelta(seconds=time_tuple[-1]))
    return proper_date

print(to_datetime(s))

dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(dt_string)

2020-12-07 17:03:03
2020-12-26 18:12:08


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

['RT @OliviaTroye: A bomb went off on US soil on Christmas morning. People are struggling to feed their families. Everyday people are saying…',
 'RT @VABVOX: If you were looking for the worst people in Congress, they are:\n▪Marsha Blackburn \n▪Ted Cruz\n▪Ron Johnson\n▪Mike Lee\n▪Rand Paul…',
 'RT @TDFORCE_Poland: Operation #Zumbach \n#Polish soldiers with @DefenceHQ and @kent_police getting #LorryDrivers thru #Covid testing.\n@teryt…',
 "RT @herbivore_club: The current pandemic shows that we can 'no longer ignore the risks' animal farming poses to public health. \n\n#COVID19 #…",
 'RT @OliviaTroye: A bomb went off on US soil on Christmas morning. People are struggling to feed their families. Everyday people are saying…']

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

['Breaking : JPOST: Tel Aviv University research says 99.9% of COVID-19 germs dead in 30 seconds with UV LEDs 🤔… https://t.co/NZ7JYp78na',
 'So just a heads up on my absence on #Twitch the past few days. I tested positive for #COVID on XMas Eve. Thankfully… https://t.co/aYiIQ9LEa5',
 "The current pandemic shows that we can 'no longer ignore the risks' animal farming poses to public health.… https://t.co/3qx0Psev2A",
 'How China Has Been Using Huawei-Made Cameras to Spy on the African Union Headquarters | #Alvarism #COVID #TLBHeart… https://t.co/TLFb0qdZ7D',
 'Thread #covid #1InA1000 https://t.co/3L4Xk8UZ2Q']

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [15]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [16]:
users_locs

[['FuelOnline',
  'Boston, MA',
  'Breaking : JPOST: Tel Aviv University research says 99.9% of COVID-19 germs dead in 30 seconds with UV LEDs 🤔… https://t.co/NZ7JYp78na'],
 ['itsBaldyKong',
  '',
  'So just a heads up on my absence on #Twitch the past few days. I tested positive for #COVID on XMas Eve. Thankfully… https://t.co/aYiIQ9LEa5'],
 ['herbivore_club',
  'Desert Island',
  "The current pandemic shows that we can 'no longer ignore the risks' animal farming poses to public health.… https://t.co/3qx0Psev2A"],
 ['AlvarismHq',
  'Washington, DC',
  'How China Has Been Using Huawei-Made Cameras to Spy on the African Union Headquarters | #Alvarism #COVID #TLBHeart… https://t.co/TLFb0qdZ7D'],
 ['susanbhalpern',
  'Hungarian Village, Columbus',
  'Thread #covid #1InA1000 https://t.co/3L4Xk8UZ2Q']]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [18]:
tweet_df

,user,location,tweet
0,FuelOnline,"Boston, MA",Breaking : JPOST: Tel Aviv University research...
1,itsBaldyKong,,So just a heads up on my absence on #Twitch th...
2,herbivore_club,Desert Island,The current pandemic shows that we can 'no lon...
3,AlvarismHq,"Washington, DC",How China Has Been Using Huawei-Made Cameras t...
4,susanbhalpern,"Hungarian Village, Columbus",Thread #covid #1InA1000 https://t.co/3L4Xk8UZ2Q


## Working With The Twitter Streaming API

With Streaming data we need to do the following:
1. Create database connection
2. Create a persistent connection to the Twitter API, and read each connection incrementally.
3. Process tweets quickly, store them in the database, and don’t let your program get backed up.
4. Handle errors and other issues properly.

Connect to MySql Database

In [19]:
#                           (server,MySQL username,MySQL pass, Database name).
connection = MySQLdb.connect("localhost","root","root","social_analytic")
cur = connection.cursor()

StreamListener Class

In [20]:
from tweepy import Stream
from tweepy.streaming import StreamListener
import tweepy
import time
import json
import sys


search_val="covid"

class listener(StreamListener):

    def on_data(self, data):
       
        try:
#             tweet_count=0
            
            all_data = json.loads(data)
            
#             search_val="covid"
            tweet_id=all_data["id"]
            created_at=to_datetime(all_data["created_at"])
            created_at_date=created_at.date()
            created_at_hour=created_at.time().strftime("%H")
            created_at_min=created_at.time().strftime("%H:%M")
            text = all_data["text"]
            source = all_data["source"]
#             local_created_at=created_at.str.replace(tzinfo=timezone.utc).astimezone(tz=None)
            local_created_at=all_data["created_at"]
            
            in_reply_to_status_id=all_data['in_reply_to_status_id_str']
            in_reply_to_user_id=all_data['in_reply_to_user_id_str']
            in_reply_to_screen_name=all_data['in_reply_to_screen_name']
            username=all_data['user']['name']
            user_screen_name=all_data['user']['screen_name']
            user_location=all_data['user']['location']
            user_url=all_data['user']['url']
            user_description=all_data['user']['description']
            user_verified_str=all_data['user']['verified']
            user_verified=0
            if user_verified_str==True:
                user_verified=1
            user_followers_count=all_data['user']['followers_count']
            user_friends_count=all_data['user']['friends_count']
            user_listed_count=all_data['user']['listed_count']
            user_favourites_count=all_data['user']['favourites_count']
            user_created_at=to_datetime(all_data['user']['created_at'])
            user_created_at_date=user_created_at.date()
            user_id=all_data['user']['id']
            user_profile_image_url_https=all_data['user']['profile_image_url_https']
            
#             coordinates_lat=all_data['coordinates'][0]
#             coordinates_lon=all_data['coordinates'][1]
            if all_data['coordinates']==True:
                coordinates_lat=all_data['coordinates'][0]
                coordinates_lon=all_data['coordinates'][1]
            else:
                coordinates_lat=None
                coordinates_lon=None
            if all_data['place']==True:
                place_country=all_data['place_country']
                place_country_code=all_data['place_country_code']
                place_full_name=all_data['place_full_name']
                place_id=all_data['place_id']
                place_type=all_data['place_type']
            else:
                place_country=None
                place_country_code=None
                place_full_name=None
                place_id=None
                place_type=None
                
            retweeted_status=None
            
            if all_data['is_quote_status']==True:
                quoted_status_id=all_data['quoted_status_id']
                quoted_status=all_data['quoted_status']
#                 retweeted_status=all_data['retweeted_status']
                quote_count=0
            else:
                quoted_status_id=None
                quote_count=0
                quoted_status=None
#                 retweeted_status=None
            try:
                reply_count=all_data['reply_count']
            except:
                reply_count=0

            retweet_count=all_data['retweet_count']
            favorite_count=all_data['favorite_count']
            retweeted_txt=all_data['retweeted']
            entities=str(all_data['entities'])
            retweeted=0
            if retweeted_txt==True:
                retweeted=1
#             feeds_link=all_data['https://twitter.com/_/status/'+id]
            feeds_link=None
            lang=all_data['lang']
            feeds_image=None
            feeds_video=None
            # clean text
            text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
            clean_text=text_preprocess(text)
            clean_text=re.sub('@[^\s]+','',clean_text)
            clean_text=re.sub(r"http\S+", "", clean_text)
            # end clean text
            text_sentiment_polarity=TextBlob(clean_text).sentiment[0]
            text_sentiment_subjectivity=TextBlob(clean_text).sentiment[1]


            cur.execute("INSERT INTO tweet (search_val,  created_at, created_at_date,created_at_hour,created_at_min, tweet_id,  text, source,in_reply_to_status_id, in_reply_to_user_id,  in_reply_to_screen_name,  user_name,  user_screen_name,  user_location, user_url,  user_description, user_verified, user_followers_count,  user_friends_count, user_listed_count,  user_favourites_count,  user_created_at,user_created_at_date, user_id,  user_profile_image_url_https,  coordinates_lat,  coordinates_lon, place_country,  place_country_code,  place_full_name, place_id,  place_type,  quoted_status_id,  quoted_status, retweeted_status,  quote_count,  reply_count,  retweet_count,  favorite_count, retweeted,  entities,  lang, feeds_link,  feeds_video, feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity) VALUES (%s, %s,%s, %s,%s, %s,%s,%s,%s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", 
                        (search_val,created_at,created_at_date,created_at_hour,created_at_min,tweet_id,text,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,username,user_screen_name,user_location,user_url,user_description,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_created_at,user_created_at_date,user_id,user_profile_image_url_https,coordinates_lat,coordinates_lon,place_country,place_country_code,place_full_name,place_id,place_type,quoted_status_id,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,retweeted,entities,lang,feeds_link,feeds_video,feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity))

            connection.commit()
            print("Fetched tweets ...........\n")
            print(created_at,user_created_at)
#             tweet_count +=1

            return True

# Catch all the errors from Database or Twiteer
        except tweepy.error.TweepError:
            print("Error fetching the data")
            
        except UnicodeEncodeError:
            pass
        
        except MySQLdb.DataError as e:
            print("DataError")
            print(e)

        except MySQLdb.InternalError as e:
            print("InternalError")
            print(e)

        except MySQLdb.IntegrityError as e:
            print("IntegrityError")
            print(e)

        except MySQLdb.OperationalError as e:
            print("OperationalError")
            print(e)

        except MySQLdb.NotSupportedError as e:
            print("NotSupportedError")
            print(e)

        except MySQLdb.ProgrammingError as e:
            print("ProgrammingError")
            print(e)
        except:
            print ("Unexpected general error: ", sys.exc_info()[0])
            raise
        finally:
            print("  ")
            #             cur.close()
            #             connection.close()
            

    def on_error(self, status):
        print (status)

In [21]:
twitterStream = Stream(auth, listener())
twitterStream.filter(track=['covid'],languages = ["en"], stall_warnings = True)

Fetched tweets ...........

2020-12-26 15:12:06 2018-02-01 23:52:58
  
Fetched tweets ...........

2020-12-26 15:12:06 2018-12-23 15:15:26
  
Fetched tweets ...........

2020-12-26 15:12:06 2020-03-13 14:11:29
  
  
Fetched tweets ...........

2020-12-26 15:12:06 2016-08-01 16:29:16
  
  
  
Fetched tweets ...........

2020-12-26 15:12:07 2019-01-15 14:55:57
  
  
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:07 2008-08-05 20:35:25
  
  
Fetched tweets ...........

2020-12-26 15:12:07 2016-09-08 17:34:32
  
Fetched tweets ...........

2020-12-26 15:12:07 2020-05-10 07:05:47
  
  
  
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:08 2013-04-16 16:26:29
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:08 2009-05-26 16:09:50
  
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:09 2014-02-01 12:09:18
  
  
Fetched tweets ...........

2020-12-26 15:12:09 2010-12-27 03:46:35
  
  
Fetched tweets ...........

2020-12-26 15:12:09 2020-08-06 15:10:51

Fetched tweets ...........

2020-12-26 15:12:23 2017-03-17 05:09:51
  
Fetched tweets ...........

2020-12-26 15:12:23 2008-05-05 02:25:44
  
Fetched tweets ...........

2020-12-26 15:12:23 2009-10-20 15:31:43
  
Fetched tweets ...........

2020-12-26 15:12:24 2009-06-30 14:14:18
  
  
Fetched tweets ...........

2020-12-26 15:12:24 2011-11-15 17:52:35
  
  
Fetched tweets ...........

2020-12-26 15:12:24 2011-12-28 03:18:01
  
Fetched tweets ...........

2020-12-26 15:12:24 2019-01-05 12:00:28
  
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:24 2013-06-01 23:57:00
  
Fetched tweets ...........

2020-12-26 15:12:24 2011-06-07 00:03:52
  
Fetched tweets ...........

2020-12-26 15:12:24 2011-04-11 18:04:56
  
  
Fetched tweets ...........

2020-12-26 15:12:24 2012-03-25 19:06:33
  
Fetched tweets ...........

2020-12-26 15:12:25 2013-09-29 18:58:36
  
  
Fetched tweets ...........

2020-12-26 15:12:25 2015-02-09 01:54:57
  
  
  
Fetched tweets ...........

2020-12-26 15:12

Fetched tweets ...........

2020-12-26 15:12:38 2010-08-31 15:14:28
  
  
  
Fetched tweets ...........

2020-12-26 15:12:38 2010-08-29 10:43:27
  
Fetched tweets ...........

2020-12-26 15:12:38 2019-02-08 17:21:43
  
  
Fetched tweets ...........

2020-12-26 15:12:38 2013-10-04 06:30:09
  
  
Fetched tweets ...........

2020-12-26 15:12:38 2013-04-16 01:21:13
  
  
  
Fetched tweets ...........

2020-12-26 15:12:39 2020-10-17 18:56:07
  
  
  
Fetched tweets ...........

2020-12-26 15:12:39 2014-08-24 04:59:21
  
Fetched tweets ...........

2020-12-26 15:12:39 2013-08-13 17:27:29
  
  
Fetched tweets ...........

2020-12-26 15:12:39 2019-09-03 22:18:13
  
  
Fetched tweets ...........

2020-12-26 15:12:39 2011-02-06 17:06:12
  
  
Fetched tweets ...........

2020-12-26 15:12:39 2010-01-03 23:19:48
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:39 2012-05-30 00:34:11
  
Fetched tweets ...........

2020-12-26 15:12:40 2013-01-22 23:40:00
  
Fetched tweets ...........

2020-12

Fetched tweets ...........

2020-12-26 15:12:56 2011-10-19 23:32:13
  
  
Fetched tweets ...........

2020-12-26 15:12:56 2012-11-09 19:25:48
  
Fetched tweets ...........

2020-12-26 15:12:56 2009-05-04 12:51:49
  
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:56 2020-11-23 21:06:41
  
  
  
Fetched tweets ...........

2020-12-26 15:12:57 2011-03-17 19:58:01
  
Fetched tweets ...........

2020-12-26 15:12:57 2014-04-19 06:41:58
  
Fetched tweets ...........

2020-12-26 15:12:57 2020-08-06 17:36:19
  
Fetched tweets ...........

2020-12-26 15:12:57 2009-05-01 18:04:49
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:57 2009-03-31 02:33:05
  
  
Fetched tweets ...........

2020-12-26 15:12:57 2009-03-07 04:37:33
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:12:57 2012-01-28 01:07:45
  
  
Fetched tweets ...........

2020-12-26 15:12:57 2008-10-30 00:42:47
  
Fetched tweets ...........

2020-12-26 15:12:57 2011-01-31 22:41:20
  
  
  
Fetched tweets .....

KeyError: 'quoted_status'